In [ ]:
import os
import pandas as pd
import numpy as np


def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna(subset=['Voltage.1', 'Time2', 'Voltage.2'])
    
    if df.empty:
        print("wrong")
        return None
    return df

def add_noise(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

def augment_data(df, augment_factor=3):
    augmented_data = [df]
    for _ in range(augment_factor - 1):
        noisy_data = add_noise(df.values)
        augmented_data.append(pd.DataFrame(noisy_data, columns=df.columns))
    return pd.concat(augmented_data, ignore_index=True)

def split_data_into_groups(data, num_groups=50):
    split_data = np.array_split(data, num_groups)
    return split_data

def save_split_data(data_splits, file_prefix, parent_folder):
    sub_folder = os.path.join(parent_folder, file_prefix)
    os.makedirs(sub_folder, exist_ok=True)
    for i, data in enumerate(data_splits):
        file_path = os.path.join(sub_folder, f"{file_prefix}_{i+1}.csv")
        data.to_csv(file_path, index=False)

data_path =  # 输入数据文件夹路径
output_folder =   # 输出数据文件夹路径
os.makedirs(output_folder, exist_ok=True)

file_names = [f for f in os.listdir(data_path) if f.endswith('.csv')]

for file_name in file_names:
    file_path = os.path.join(data_path, file_name)
    print(f"正在处理文件: {file_name}")
    df = load_data(file_path)
    
    if df is None:
        continue
    
    augmented_df = augment_data(df)
    print(f"增强后的数据形状: {augmented_df.shape}")
    
    split_data = split_data_into_groups(augmented_df, num_groups=50)
    
    save_split_data(split_data, file_name.split('.')[0], output_folder)